In [1]:
!pip uninstall scipy
!pip install scipy==1.1.0

Uninstalling scipy-1.4.1:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/scipy-1.4.1.dist-info/*
    /usr/local/lib/python3.6/dist-packages/scipy/*
Proceed (y/n)? y
  Successfully uninstalled scipy-1.4.1
     |████████████████████████████████| 31.2MB 108kB/s 
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


In [2]:
from PIL import Image
from skimage import io, feature, color, util
import os
import sys
import numpy as np
import torch
from torch.autograd import Variable
from torch.utils.data import DataLoader, SequentialSampler
import torchvision.transforms as transforms
from torchvision.utils import save_image

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
sys.path.append('/content/gdrive/MyDrive/src_first')

In [5]:
from models import InpaintGenerator
from data import get_training_set, get_test_set, get_val_set, create_iterator
from dataset import DatasetFromFolder
from util import is_image_file, load_img, save_img

In [6]:
root = '/content/gdrive/MyDrive'
dataset = 'dataset'
model = '/content/gdrive/MyDrive/checkpoint_first/netG_weights_epoch_5.pth'
cuda = True

In [7]:
root_path = root
val_set = get_val_set(os.path.join(root_path , dataset))
seq_sampler = SequentialSampler(val_set)
val_data_loader = DataLoader(dataset=val_set, num_workers=0, batch_size=1, shuffle=False,sampler = seq_sampler)

In [8]:
checkpoint = torch.load(model)
netG = InpaintGenerator()
netG.load_state_dict(checkpoint['generator'])
netG.cuda()

InpaintGenerator(
  (encoder): Sequential(
    (0): ReflectionPad2d((3, 3, 3, 3))
    (1): Conv2d(1, 64, kernel_size=(7, 7), stride=(1, 1))
    (2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (3): ReLU(inplace=True)
    (4): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (5): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (8): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (9): ReLU(inplace=True)
  )
  (middle): Sequential(
    (0): ResnetBlock(
      (conv_block): Sequential(
        (0): ReflectionPad2d((2, 2, 2, 2))
        (1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=(2, 2))
        (2): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (3): ReLU(inplace=True)
 

In [9]:
transform_list = [transforms.ToTensor(),
                  transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
                  
transform = transforms.Compose(transform_list)

In [10]:
counter = 0

with torch.no_grad():
    for batch in val_data_loader:
        input, target = Variable(batch[0], volatile=True), Variable(batch[1], volatile=True)
        if cuda:
            input = input.cuda()
            target = target.cuda()


        pred_input = input
        out = netG(pred_input)
        tmp = out
        
        results_dir = root_path + '/results_first'
        if not os.path.exists(results_dir):
            os.makedirs(results_dir)
       
        image_name = dataset + "_" + str(counter).zfill(5) + ".jpg"
        save_image(out, results_dir + "/{}".format(image_name))
        print("saving:"+image_name)
        
        counter += 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  """


saving:dataset_00000.jpg
saving:dataset_00001.jpg
saving:dataset_00002.jpg
